# AI usecase decision tree optimization

## Processing

### Process the answers

#### Group identical nodes

In [1]:
def group_identical_answer_nodes(tree):
    grouped_answers = {}
    replace_by = []
    to_modify = set()
    new_tree = []

    for item in tree:
        if item['type'] == 'answer':
            output = item['content']
            for source in item['source']:
                output += ", " + source

            if output not in grouped_answers:
                grouped_answers[output] = []
            grouped_answers[output].append(item)

    for sources, items in grouped_answers.items():
        first_item = items[0]
        deleted_nodes = [item['id'] for item in items[1:]]

        in_values = []
        for item in items:
            if 'in' in item:
                in_values.extend(item['in'] if isinstance(item['in'], list) else [item['in']])
        to_modify.update(in_values)

        first_item['in'] = in_values

        replace_by.append({
            "id": first_item['id'],
            "deleted_nodes": deleted_nodes
        })

        new_tree.append(first_item)

    # Add untouched nodes
    for item in tree:
        if item['type'] != 'answer':
            new_tree.append(item)

    return new_tree, replace_by, list(to_modify)


#### Update parent nodes

In [2]:
def update_question_nodes(tree, replace_by, to_modify):
    updated_tree = []

    for item in tree:
        if item['type'] == 'question':
            if item['id'] in to_modify:
                updated_item = item.copy()

                for mapping in replace_by:
                    if updated_item['yes'] in mapping['deleted_nodes']:
                        updated_item['yes'] = mapping['id']
                        break

                for mapping in replace_by:
                    if updated_item['no'] in mapping['deleted_nodes']:
                        updated_item['no'] = mapping['id']
                        break

                updated_tree.append(updated_item)
            else:
                updated_tree.append(item)
        else:
            updated_tree.append(item)

    return updated_tree


### Process the question nodes

#### Group identical nodes

In [3]:
def group_identical_question_nodes(tree, to_modify):
    grouped_questions = {}
    question_replace_by = []
    question_to_modify = set()
    updated_tree = []
    processed = []

    # Collect question nodes from new_tree
    question_nodes = [node for node in tree if node['type'] == 'question' and node['id'] in to_modify]

    # Group question nodes by (content, yes, no)
    for item in question_nodes:
        key = (item['content'], item['yes'], item['no'])
        if key not in grouped_questions:
            grouped_questions[key] = []
        grouped_questions[key].append(item)

    # Process groups
    for key, items in grouped_questions.items():
        if len(items) > 1:
            first_item = items[0]
            deleted_nodes = [item['id'] for item in items[1:]]

            # Merge 'in' values
            in_values = []
            for item in items:
                if 'in' in item:
                    in_values.extend(item['in'] if isinstance(item['in'], list) else [item['in']])

            question_to_modify.update(in_values)

            # Update 'in' of surviving node
            first_item['in'] = in_values
            
            # Add to question_replace_by mapping
            question_replace_by.append({
                "id": first_item['id'],
                "deleted_nodes": deleted_nodes
            })

            updated_tree.append(first_item)
        # else:
        #     updated_tree.append(items[0])

    # # Add untouched question nodes
    # for item in tree:
    #     if item['type'] == 'question' and item['id'] not in [q['id'] for group in grouped_questions.values() for q in group]:
    #         updated_tree.append(item)

    # # Add answer nodes
    # for item in tree:
    #     if item['type'] == 'answer':
    #         updated_tree.append(item)

    # processed = []
    for q in question_replace_by:
        processed.append(q['id'])
        processed.extend(q['deleted_nodes'])

    return updated_tree, question_replace_by, list(question_to_modify), processed


#### Update parent nodes

In [4]:
def update_remaining_question_nodes(tree, new_tree, question_replace_by, question_to_modify, processed):
    updated_tree = new_tree

    for item in tree:

        # Check if this question node's id is in question_to_modify
        if item['id'] in question_to_modify:
            updated_item = item.copy()

            # Update 'yes' value if needed
            for mapping in question_replace_by:
                if updated_item['yes'] in mapping['deleted_nodes']:
                    updated_item['yes'] = mapping['id']
                    break

            # Update 'no' value if needed
            for mapping in question_replace_by:
                if updated_item['no'] in mapping['deleted_nodes']:
                    updated_item['no'] = mapping['id']
                    break

            # Add updated question node
            updated_tree.append(updated_item)
        # Node is not affected, just copy
        elif item['id'] not in processed:
            updated_tree.append(item)

    return updated_tree

### Iteration over the tree

In [5]:
def reduce_tree(tree):
    print("Starting reduction...")    
    print("current node count: ", len(tree))

    # Step 1: Merge identical answers and update parents
    new_tree, replace_by, to_modify = group_identical_answer_nodes(tree)
    new_tree = update_question_nodes(new_tree, replace_by, to_modify)

    # Save original tree
    tree = new_tree

    while True:
        print("current node count: ", len(tree))

        # Step 2: Merge identical questions layer by layer
        new_tree, question_replace_by, question_to_modify, processed = group_identical_question_nodes(new_tree, to_modify)
        new_tree = update_remaining_question_nodes(tree, new_tree, question_replace_by, question_to_modify, processed)

        if len(new_tree) == len(tree):
            break  # No further reduction
        else:
            # Move to next row
            to_modify = question_to_modify 
            tree = new_tree

    print("Reduction complete.")
    return new_tree


## Implementation

### Load the data

In [6]:
import json

# Load the PDF paths from the JSON file
with open('data/sources/tree.json', 'r') as file:
    tree = json.load(file)

### Process the data

In [7]:
final_tree = reduce_tree(tree)

Starting reduction...
current node count:  95
current node count:  65
current node count:  61
Reduction complete.


### Save the new tree

In [8]:
# Save the new_tree structure to a JSON file
with open('data/output/final_tree.json', 'w') as json_file:
    json.dump(final_tree, json_file, ensure_ascii=False)

print(f"Data saved to data/output/final_tree.json")

Data saved to data/output/final_tree.json


## Exploration

### Load the trees

In [9]:
with open('data/output/final_tree.json', 'r') as file:
    optimized_tree = json.load(file)

with open('data/sources/tree.json', 'r') as file:
    original_tree = json.load(file)

In [10]:
# Preprocess the tree: {id: node}
opti_dict = {node['id']: node for node in optimized_tree}
og_dict = {node['id']: node for node in original_tree}


### Find the root

In [11]:
root = tree[0]
for node in tree:
    if node['in'] == []:
        root = node
        break
root['id']

'b26'

### Exploring functions

#### Explore a specific path

In [12]:
def explore_custom_path(start_id, tree_dict, path_choices):
    current_id = start_id
    path = []
    choice_index = 0

    while True:
        node = tree_dict.get(current_id)

        if not node:
            print(f"Node with id {current_id} not found!")
            break

        path.append(node)

        if node['type'] == 'answer':
            print(f"Reached answer node: {node['id']}")
            break

        # Determine which branch to follow
        if choice_index >= len(path_choices):
            print(f"No more choices left, stopping at node {node['id']}")
            break

        choice = path_choices[choice_index]
        if choice not in ["yes", "no"]:
            print(f"Invalid choice '{choice}' at step {choice_index}, stopping.")
            break

        current_id = node[choice]
        choice_index += 1

    return path


#### Explore all possible paths

In [13]:
def explore_all_paths(start_id, tree_dict):
    all_paths = []

    def dfs(current_id, current_path):
        node = tree_dict.get(current_id)
        if not node:
            print(f"Node with id {current_id} not found!")
            return

        # Add current node to the path
        current_path.append(node)

        # If answer node, record path
        if node['type'] == 'answer':
            all_paths.append(list(current_path))  # Copy of current_path
        else:
            # Recurse on "yes" branch
            dfs(node['yes'], list(current_path))  # Copy to avoid mutation

            # Recurse on "no" branch
            dfs(node['no'], list(current_path))

    dfs(start_id, [])
    return all_paths


#### Display a path

In [14]:
def display_path(path):
    print("Explored Path:")
    for node in path:
        if node['type'] == 'question':
            print(f"Node ID: {node['id']}, Content: {node['content']}")
        if node['type'] == 'answer':
            print(f"Node ID: {node['id']}, Content: {node['content']}, Sources: {node['source']}")

### Explore the optimized tree

#### Specific path

In [15]:
custom_path = ["yes", "no", "yes", "yes"]
opti_test_path = explore_custom_path(root['id'], opti_dict, custom_path)

display_path(opti_test_path)

Reached answer node: g2
Explored Path:
Node ID: b26, Content: S'agit-il d'une rÃ©vision sur demande ou d'une rÃ©vision d'office?
Node ID: c26, Content: Droit ouvert dans le systÃ¨me linÃ©aire ?
Node ID: e34, Content: Y a-t-il eu une modification des faits entre le 01.01.2022 et le 31.12.2023 ?
Node ID: h34, Content: L'Ã¢ge de lâ€™assurÃ©, au 01.01.2022, est-il Ã©gal ou supÃ©rieur (=>) Ã  55 ans ?
Node ID: g2, Content: Rente par pallier, Sources: ['Lettre c des dispositions transitoires de la modification du 19 juin 2020 (DÃ©veloppement continu de lâ€™AI)']


#### All paths

In [16]:
opti_all_paths = explore_all_paths(root['id'], opti_dict)

print(len(opti_all_paths), "Possible Paths")

48 Possible Paths


In [17]:
all_paths_reduced = []

for path in opti_all_paths:
    reduced_path = {'path': [], 'answer': ''}
    for i, node in enumerate(path):
        if node['type'] == 'question':
            if node['yes'] == path[i+1]['id']:
                reduced_path['path'].append('yes')
            else:
                reduced_path['path'].append('no')
        else:
            reduced_path['answer'] = f"Décision: {node['content']}, Sources: {node['source']}"
    all_paths_reduced.append(reduced_path)

# Save the new_tree structure to a JSON file
with open('data/output/all_paths.json', 'w') as json_file:
    json.dump(all_paths_reduced, json_file, ensure_ascii=False)

print(f"Data saved to data/output/all_paths.json")

Data saved to data/output/all_paths.json


### Explore the original tree

#### All paths

In [18]:
og_all_paths = explore_all_paths(root['id'], og_dict)

print(len(og_all_paths), "Possible Paths")

48 Possible Paths


### Compare outcomes

In [19]:
similarity_ratio = 0

for i, path in enumerate(og_all_paths):
    print(f"Path {i}:")

    print(f"\nOriginal")
    display_path(path)

    print(f"\nOptimized")
    display_path(opti_all_paths[i])
    print(path[-1])

    og_outcome = f"{path[-1]['content']}, source: {path[-1]['source']}"
    opti_outcome = f"{opti_all_paths[i][-1]['content']}, source: {opti_all_paths[i][-1]['source']}"

    if og_outcome == opti_outcome:
        print(f"\nSame outcome:")
        print(f"Outcome: {og_outcome}")
        similarity_ratio += 1
        
    else:
        print(f"\nDifferent outcomes:")
        print(f"Optimized outcome: {opti_outcome}")
        print(f"Original outcome: {og_outcome}")

    print("\n----------------------------------------\n")

print(f"Similarity Ratio: {similarity_ratio / len(og_all_paths) * 100}%")

Path 0:

Original
Explored Path:
Node ID: b26, Content: S'agit-il d'une rÃ©vision sur demande ou d'une rÃ©vision d'office?
Node ID: c26, Content: Droit ouvert dans le systÃ¨me linÃ©aire ?
Node ID: e26, Content: Y a-t-il eu une modification des faits entre le 01.01.2022 et le 31.12.2023 ?
Node ID: h28, Content: Le degrÃ© dâ€™invaliditÃ© sâ€™est-il modifiÃ© dâ€™au-moins 5% ?
Node ID: k30, Content: Est-il augmentÃ© ?
Node ID: n30, Content: Y a-t-il eu une augmentation du taux depuis le 01.01.2024 ?
Node ID: t30, Content: Rente linÃ©aire, Sources: ['Lettre b points 1 et 2 des dispositions transitoires de la modification du 19 juin 2020 (DÃ©veloppement continu de lâ€™AI)', 'Lettre 1 des dispositions transitoires relatives Ã\xa0 la modification du 18 octobre 2023 (RAI)']

Optimized
Explored Path:
Node ID: b26, Content: S'agit-il d'une rÃ©vision sur demande ou d'une rÃ©vision d'office?
Node ID: c26, Content: Droit ouvert dans le systÃ¨me linÃ©aire ?
Node ID: e26, Content: Y a-t-il eu une modi

In [20]:
def check_path(path):
    found = False
    for p in all_paths_reduced:
        if path == p['path']:
            print(p['answer'])
            found = True
            break
    if not found:
        print("No matching path found.")

In [21]:
check_path(["yes", "no", "yes", "yes"])

Décision: Rente par pallier, Sources: ['Lettre c des dispositions transitoires de la modification du 19 juin 2020 (DÃ©veloppement continu de lâ€™AI)']
